# Assessment: Weekly deliverable template - Regression
<font color='#AC135A'>**Applied Machine Learning** - **Sheffield Hallam University**</font>


---

## Introduction
Add your solutions to the different sub-tasks requested for this deliverable.

<font color='red'>DO NOT ADD MORE CELLS TO THE FINAL VERSION OF THIS NOTEBOOK.

### 1. Dataset loading

In [1]:
import pandas as pd

df = pd.read_csv('wines_SPA.csv')
print(df.describe())

            rating   num_reviews        price         body      acidity
count  7500.000000   7500.000000  7500.000000  6331.000000  6331.000000
mean      4.254933    451.109067    60.095822     4.158427     2.946612
std       0.118029    723.001856   150.356676     0.583352     0.248202
min       4.200000     25.000000     4.990000     2.000000     1.000000
25%       4.200000    389.000000    18.900000     4.000000     3.000000
50%       4.200000    404.000000    28.530000     4.000000     3.000000
75%       4.200000    415.000000    51.350000     5.000000     3.000000
max       4.900000  32624.000000  3119.080000     5.000000     3.000000


**Observations:**  

Comparing the csv file with the "describe()" out put, I can see that "describe" only shows numerical columns. Several categorical columns do not show. 

I can see that there are many examples in this dataset, 7,500. 

*Rating*

The rating values have a range between 4.2 and 4.9. The 75% quartile is at 4.2, the minimum. Adding the std to the mean shows that 68% of examples have a rating below 4.4, and 95% below 4.5.  This shows that this variable is very much imbalanced. The standard deviation is correspondingly small. The name suggests this is likely a target variable as the user rating is something businesses may want to predict. 

*num_reviews*

From the name I infer that this is the number of available reviews that lead to the "rating". The range is very high between 25 and 32.624. However, the 25% and 75% quartiles are in the same magnitude at 389 and 415, respectively. This shows that 50% of review numbers are in this range. The location is skewed to lower values. The standard deviation is quite high though. Only 68% of examples will have review numbers below mean+std = 1174, and 95% are below mean+2*std = 1897. (I'm ignoring the bottom because the minimum value is higher than mean-std.) This suggest we definitely can see some outliers at the top end (32,624), which we may want to handle somehow. 

*price*

We see similar behaviour of the price data: A huge range (4.99–3119.08), a relatively strong 25-75% range (18.9-51.35) that is located at the lower proce range, and again a very high standard deviation that shows that only 68% of examples have a price below 210.45, while 95% have a price below 360.81. The expectation again is to have some outliers at the top end. It may be an interesting question if the price correlates to rating in some way. Also, the expectation is that the number of reviews is higher towards the lower end of the price range. 

*body and acidity*

The count is lower than for the previous three attributes (6331 instead of 7500). This means there are a significant number of missing values. If we use body as a feature this is something we need to take into consideration. As for the values, the range for body is between 2.0 and 5.0, and we don't see any decimals. This could mean this body is likely determined as a rating between 1 and 5. For acidity the range is even smaller: 1.0-3.0. 95% of examples have ratings for body and acity of 3 and more, or actually only 3 for acidity. This suggests that we can dismiss acidity as a feature, as there is no information in this value. We'll need to also be critical about body as a feature. 








### 2. At least two exploratory graphs with captions



In [ ]:
# ---- Your code here ----

**Observations:**  

### 3. Histogram showing rating imbalance (if any)


In [ ]:
# ---- Your code here ----

**Observations:**  

### 4. Price-regression model + metrics + plot


In [ ]:
# ---- Your code here ----

**Observations:**  

### 5. Quality-regression model + metrics + plot


In [ ]:
# ---- Your code here ----

**Observations:**  

### 6. Over/under-fitting discussion and fix

In [ ]:
# ---- Your code here ----

**Observations:**  

### 7. Apply cross-validation or statistical analyses on the features or results of the regression models (advanced)

In [ ]:
# ---- Your code here ----

**Observations:**  